# Proyecto de Inteligencia Artificial

**Autores:** 
- Ian Coto 
- Rafael Vargas

---

## 1. Fase de entendimiento del negocio

### i. Objetivos del negocio

El objetivo principal es **analizar los datos de pitcheo de la MLB (Major League Baseball) del periodo 2015-2024** para entender patrones, tendencias y características que permitan:

1. **Identificar factores clave del desempeño de pitchers**: Analizar qué variables están más correlacionadas con el éxito en el pitcheo.
2. **Predecir resultados de lanzamientos**: Desarrollar modelos que puedan anticipar el resultado de un pitch basándose en características del lanzamiento.
3. **Clasificar estilos de pitcheo**: Agrupar pitchers según sus características y estrategias de lanzamiento.
4. **Optimizar estrategias**: Proporcionar insights que puedan ayudar a equipos y jugadores a mejorar su rendimiento.

### ii. Criterios de éxito (en términos del negocio)

1. **Comprensión profunda de los datos**: Identificar al menos 5 variables clave que impactan significativamente los resultados.
2. **Modelos predictivos precisos**: Alcanzar una precisión mínima del 70% en la predicción de resultados de lanzamientos.
3. **Segmentación clara**: Identificar al menos 3-5 clusters distintos de estilos de pitcheo con características interpretables.
4. **Insights accionables**: Generar recomendaciones basadas en datos que puedan ser aplicadas por equipos y jugadores.

---

## 2. Fase de entendimiento de los datos

### i. Objetivos de minería de datos

1. **Análisis exploratorio exhaustivo**: Realizar un análisis estadístico completo de todas las variables relevantes del dataset.
2. **Identificación de patrones**: Descubrir relaciones entre variables y tendencias temporales en los datos.
3. **Detección de anomalías**: Identificar valores atípicos y datos de baja calidad que requieran limpieza.
4. **Feature engineering**: Crear nuevas variables derivadas que puedan mejorar los modelos predictivos.
5. **Preparación de datos**: Limpiar y transformar los datos para su uso en modelos de machine learning.

### ii. Criterios de éxito (desde la perspectiva de la minería de datos)

1. **Calidad de datos**: Lograr un dataset con menos del 5% de valores faltantes en variables clave.
2. **Visualizaciones informativas**: Crear al menos 10 visualizaciones que revelen insights importantes.
3. **Correlaciones significativas**: Identificar variables con correlación |r| > 0.3 con las variables objetivo.
4. **Dataset balanceado**: Asegurar representación adecuada de diferentes tipos de lanzamientos y resultados.
5. **Documentación completa**: Mantener registro detallado de todas las transformaciones y decisiones tomadas.

---

## 6. Conclusiones

### Hallazgos de Exploración de Datos:

1. **Dataset robusto**: El dataset de Statcast proporciona información pitch-by-pitch extremadamente detallada con más de 90 variables que describen cada aspecto del lanzamiento.

2. **Variables clave identificadas**:
   - **Velocidad (release_speed)**: Muestra distribución normal con media ~92 mph
   - **Tipos de lanzamiento (pitch_type)**: Four-Seam Fastball y Slider son los más comunes
   - **Ubicación (zone, plate_x, plate_z)**: Permite análisis detallado de la estrategia de pitcheo
   - **Resultados (events)**: Captura todos los posibles desenlaces de un at-bat

3. **Calidad de datos**: 
   - Algunas variables tienen valores nulos significativos (especialmente métricas de bateo)
   - Esto es esperado ya que no todos los pitches resultan en contacto
   - No se encontraron duplicados significativos

4. **Correlaciones interesantes**: Variables relacionadas con la física del lanzamiento muestran correlaciones fuertes entre sí (velocidad inicial, aceleración, etc.)

5. **Outliers presentes**: Detectados en varias variables, pero muchos son valores legítimos (ej: velocidades extremadamente altas o bajas para ciertos tipos de pitch)

### Resultados de Modelos Predictivos:

#### **Notebook 04: Predicción de Tipos de Lanzamiento**

6. **Modelo 1: Random Forest para Predicción de Tipos de Lanzamiento (18 clases)**:
   - **Dataset procesado**: 1,534,286 lanzamientos (2023-2024), 63 variables
   - **Accuracy**: 90.92% (supera ampliamente el objetivo del 70%)
   - **F1-Score Weighted**: 90.82% | **F1-Score Macro**: 78.97%
   - **Tamaño del modelo**: 100 árboles, entrenado con ~1.2M ejemplos
   
   **Resultados por tipo de pitch**:
   - **Excelente desempeño (F1 > 0.90)**: UN (99.4%), FF (96.8%), KN (95.5%), SI (93.8%), CH (93.3%)
   - **Buen desempeño (F1: 0.70-0.89)**: CU (88.0%), EP (88.8%), SC (87.1%), SL (85.9%), ST (85.4%), FS (81.8%), FC (79.4%), KC (77.4%)
   - **Desempeño problemático (F1 < 0.70)**: SV (69.4%), FO (58.3%), CS (51.3%), **PO (0% - fallo total)**
   
   **Problema crítico detectado**: 
   - Overfitting severo: 100% accuracy en entrenamiento vs 90.92% en prueba (diferencia de 9.08%)
   - Clases minoritarias completamente ignoradas (PO: 21 casos, 0% F1-score)

#### **Notebook 05: Predicción de Resultados del Lanzamiento**

7. **Modelo 2: Random Forest para Clasificación Ball/Strike/In Play (3 clases)**:
   - **Dataset**: 1,534,286 lanzamientos con 16 features
   - **Accuracy**: 70.76% (cumple objetivo del 70% por margen mínimo)
   - **F1-Score Weighted**: 0.67 | **F1-Score Macro**: 0.59
   
   **Resultados por clase**:
   - **Ball (B)**: F1=0.83 (precision=0.79, recall=0.86) - Muy bueno, 110,594 casos
   - **Strike (S)**: F1=0.73 (precision=0.67, recall=0.80) - Aceptable, 142,340 casos
   - **In Play (X)**: F1=0.22 (precision=0.48, recall=0.14) - **FALLO CRÍTICO**, 53,924 casos
   
   **Variables más importantes**: 
   - zone (18%), plate_x (15%), plate_z (15%), release_spin_rate (8%), pfx_x (8%), pfx_z (8%)
   
   **Problema crítico**: El modelo predice correctamente Ball/Strike pero casi nunca predice "In Play" (recall 14%)

#### **Notebook 06: Predicciones Avanzadas**

8. **Modelo 3: Random Forest para Predicción del Próximo Lanzamiento (5 clases principales)**:
   - **Dataset con feature engineering**: Variables de tendencia (lag features) y contexto del juego
   - **Accuracy**: 50% (muy por debajo del objetivo del 70%)
   - **F1-Score Weighted**: 0.49 | **F1-Score Macro**: 0.50
   - **Tamaño de test**: 245,402 lanzamientos
   
   **Resultados por clase**:
   - **FF (Four-Seam)**: F1=0.44 (precision=0.75, recall=0.31) - Alta precisión pero baja recall
   - **SI (Sinker)**: F1=0.58 (precision=0.57, recall=0.59) - Mejor balance
   - **SL (Slider)**: F1=0.51 (precision=0.46, recall=0.56)
   - **FC (Cutter)**: F1=0.51 (precision=0.38, recall=0.75)
   - **CH (Changeup)**: F1=0.47 (precision=0.37, recall=0.62)
   
   **Problema crítico**: Predecir comportamiento futuro del pitcher es inherentemente difícil, depende de estrategia/psicología no capturada por las variables

9. **Modelo 4: Random Forest para Probabilidad de Swing (2 clases)**:
   - **Accuracy**: 75.11% (buen desempeño, supera el 70%)
   - **Variables más importantes**: 
     - plate_x (37%) - Ubicación horizontal dominante
     - plate_z (32%) - Ubicación vertical
     - strikes (17%) - Cuenta del bateador
     - balls (6%)
   
   **Insight clave**: La ubicación del pitch (plate_x + plate_z = 69%) es el factor dominante para predecir si el bateador hará swing

#### **Notebook 07: Clustering de Estilos de Pitcheo**

10. **Modelo 5: K-Means Clustering para Identificación de Arquetipos (No Supervisado)**:
   - **Dataset agregado**: 938 pitchers con > 100 lanzamientos cada uno
   - **Variables utilizadas**: avg_speed, avg_spin, std_pfx_x, std_pfx_z, fastball_pct, breaking_pct, offspeed_pct
   - **Método**: K-Means con k=4 clusters, normalización con StandardScaler
   - **Reducción dimensional**: PCA con 2 componentes (varianza explicada visible en visualización)
   
   **Arquetipos identificados** (4 clusters):
   - **Cluster 0 - Lanzadores Equilibrados**: avg_speed=87.9 mph, avg_spin=2077, fastball=50%, breaking=21%, offspeed=27%
   - **Cluster 1 - Especialistas en Rompientes (High Break)**: avg_speed=88.4 mph, avg_spin=2323, fastball=54%, breaking=38%, std_pfx_x alto (0.996)
   - **Cluster 2 - Especialistas en Rompientes (High Movement)**: avg_speed=89.2 mph, avg_spin=2365, fastball=50%, breaking=42%, std_pfx_z alto (0.783)
   - **Cluster 3 - Power Pitchers (Fastball Heavy)**: avg_speed=91.4 mph, avg_spin=2231, **fastball=71%**, breaking=18%, offspeed=9%
   
   **Validación del objetivo**: Identificados 4 clusters distintos con características interpretables (cumple objetivo de 3-5 clusters)

### Problemas Generales Identificados y Soluciones:

   **a) Overfitting Severo** (100% entrenamiento vs 90.92% prueba):
   - **Causas**: Árboles muy profundos sin regularización, parámetros permisivos
   - **Soluciones**: Limitar `max_depth`, aumentar `min_samples_split/leaf`, validación cruzada robusta

   **b) Desbalance de Clases** (PO: 21 casos, CS: 27, FO: 194):
   - **Impacto**: Clases minoritarias ignoradas completamente por el modelo
   - **Soluciones**: SMOTE, `class_weight='balanced'`, fusión de clases similares, eliminar clases con < 100 ejemplos


   **c) Confusión entre Clases Similares** (SV/SL/CU, FO/FS, CS/CU):
   - **Causas**: Características físicas muy similares, inconsistencia en etiquetación
   - **Soluciones**: Feature engineering (ratios, interacciones), modelos jerárquicos, fusión de clases

   **d) Fallo en Predicción de "In Play" (Modelo 2)** - F1=0.22:
   - **Causas**: Eventos "In Play" son raros (~17.6%) y dependen de contacto bateador-pelota, no solo del lanzamiento
   - **Impacto**: El modelo predice principalmente Ball/Strike, ignorando casi todos los In Play
   - **Soluciones**: Incorporar variables del bateador, SMOTE, modelos separados para contacto vs no-contacto

   **e) Bajo Desempeño en Predicción Secuencial (Modelo 3 - Notebook 06)** - Accuracy=50%:
   - **Causas**: Predecir comportamiento futuro del pitcher es inherentemente difícil, depende de estrategia/psicología no capturada
   - **Impacto**: Features de tendencia (lag variables) no capturan suficiente información del patrón del pitcher
   - **Soluciones**: Modelos LSTM/RNN para secuencias temporales, más contexto histórico del pitcher, análisis de tendencias por situación

   **f) Clustering No Supervisado Exitoso (Modelo 5 - Notebook 07)**:
   - **Logro**: Identificación clara de 4 arquetipos de pitchers con características distintivas
   - **Interpretabilidad**: Power Pitchers (71% fastballs), Especialistas en Rompientes (38-42% breaking), Equilibrados
   - **Aplicación práctica**: Permite clasificar nuevos pitchers y entender estilos de lanzamiento

11. **Validación de Objetivos del Proyecto**:
   
   **Por modelo individual**:
   -**Modelo 1 - Tipos de pitch (NB 04)**: 90.92% >> 70% - **Excelente**
   -**Modelo 2 - Ball/Strike/In Play (NB 05)**: 70.76% ≈ 70% - **Cumple** (con limitaciones en "In Play")
   -**Modelo 3 - Próximo lanzamiento (NB 06)**: 50% << 70% - **No cumple objetivo**
   -**Modelo 4 - Probabilidad de Swing (NB 06)**: 75.11% >> 70% - **Muy bueno**
   -**Modelo 5 - Clustering (NB 07)**: 4 clusters identificados - **Cumple objetivo** (3-5 clusters esperados)
   
   **Evaluación general del proyecto**:
   -**Variables clave identificadas**: Más de 5 variables críticas encontradas (ubicación, velocidad, spin, contexto)
   -**Modelos predictivos**: 3 de 4 modelos supervisados cumplen o superan el 70% de accuracy
   -**Segmentación de estilos**: 4 arquetipos de pitchers claramente diferenciados e interpretables
   -**Advertencia crítica**: Overfitting detectado en múltiples modelos - desempeño real podría ser menor
   -**Limitaciones identificadas**: Clases minoritarias, eventos raros y predicción secuencial tienen desempeño subóptimo
   -**Valor práctico**: 4 de 5 modelos son útiles para aplicaciones reales en análisis de béisbol
   
   **Conclusión final**: El proyecto cumple satisfactoriamente los objetivos de negocio establecidos, con modelos robustos para predicción de tipos de pitch, resultados y comportamiento del bateador, además de una segmentación clara de estilos de pitcheo.